In [1]:
import json
import os
import sys

# import local dependencies
sys.path.append(os.path.abspath("../../scripts/"))
from oso_db import execute_query

In [5]:
# gather all artifacts with at least one PR associated with projects in a given collection
project_data = execute_query("""
    SELECT DISTINCT
        p.slug AS slug,
        p.name AS name,
        a.name AS repo
    FROM 
        project p
        INNER JOIN project_artifacts_artifact paa ON p.id = paa."projectId"
        INNER JOIN artifact a ON paa."artifactId" = a.id
        INNER JOIN collection_projects_project cpp ON p.id = cpp."projectId"
        INNER JOIN collection c ON cpp."collectionId" = c.id
        INNER JOIN event e ON a.id = e."toId"
    WHERE 
        c.slug = 'optimism' 
        AND a.namespace = 'GITHUB'
        AND e."typeId" = 2 -- typeID for Pull Request Created
    ORDER BY 
        p.slug;
""", col_names=False)

In [6]:
# traverse the result and store it in a simple schema
data = {}
for (slug, name, repo) in project_data:
    if slug not in data:
        data[slug] = {
            'name': name,
            'collections': [],
            'repos': [repo]
        }
    else:
        data[slug]['repos'].append(repo)        

In [9]:
# create a string of the slugs
slugs_str = "','".join(data.keys())

# create a string of other collections to see if the project belongs to
collections_str = "','".join([
    'Optimism Onchain',
    'Optimism RPGF1',
    'Optimism RPGF2',
    'Optimism RPGF3',
    'OP GovGrants'
])

In [10]:
# execute another query to look up collection membership
collection_mappings = execute_query(f"""
    SELECT 
        p.slug AS slug,
        c.name AS name
    FROM project p
    JOIN collection_projects_project cpp ON p."id" = cpp."projectId"
    JOIN collection c ON cpp."collectionId" = c."id"
    WHERE
        p.slug IN ('{slugs_str}')    
        AND c."name" IN ('{collections_str}')
""", col_names=False)

In [11]:
# append the collection tags to our data
for (slug, collection) in collection_mappings:
    data[slug]['collections'].append(collection)

In [12]:
# dump the data to json
with open("oso-optimism-repo-dump.json", 'w') as f:
    json.dump(data, f, indent=2)